# Libraries

In [1]:
# libraries
# ----------

import requests
import pandas as pd
import wget

# Downloading data

# Dataframes

In [2]:
# dataset
# --------

conf_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('time_series_covid19_deaths_global.csv')
recv_df = pd.read_csv('time_series_covid19_recovered_global.csv')

In [3]:
conf_df.head()
# deaths_df.head()
# recv_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,56294,56322,56384,56454,56517,56572,56595,56676,56717,56779
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,124134,124419,124723,125157,125506,125842,126183,126531,126795,126936
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,116836,116946,117061,117192,117304,117429,117524,117622,117739,117879
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,11850,11888,11944,12010,12053,12115,12174,12231,12286,12328
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,22063,22132,22182,22311,22399,22467,22579,22631,22717,22885


In [4]:
#conf_df.columns
# deaths_df.columns
# recv_df.columns

In [5]:
conf_df.columns[4:]

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '3/28/21', '3/29/21', '3/30/21', '3/31/21', '4/1/21', '4/2/21',
       '4/3/21', '4/4/21', '4/5/21', '4/6/21'],
      dtype='object', length=441)

# Merging dataframes

In [6]:
# extract dates
dates = conf_df.columns[4:]

# melt dataframes in longer format
conf_df_long = conf_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Confirmed')

deaths_df_long = deaths_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Deaths')

recv_df_long = recv_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Recovered')

recv_df_long = recv_df_long[recv_df_long['Country/Region']!='Canada']

print(conf_df_long.shape)
print(deaths_df_long.shape)
print(recv_df_long.shape)

(120834, 6)
(120834, 6)
(113778, 6)


In [7]:
# merge dataframes

# full_table = pd.concat([conf_df_long, deaths_df_long['Deaths'], recv_df_long['Recovered']], 
#                        axis=1, sort=False)

full_table = pd.merge(left=conf_df_long, right=deaths_df_long, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])
full_table = pd.merge(left=full_table, right=recv_df_long, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])

full_table.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0,0,0.0
1,NaN,Albania,41.15330,20.168300,1/22/20,0,0,0.0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0,0,0.0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0,0,0.0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0,0,0.0


In [8]:
full_table.shape

(120834, 8)

In [9]:
full_table.isna().sum()

Province/State    83349
Country/Region        0
Lat                 441
Long                441
Date                  0
Confirmed             0
Deaths                0
Recovered          9261
dtype: int64

In [10]:
full_table[full_table['Recovered'].isna()]['Country/Region'].value_counts()

Canada         7056
China           882
Mozambique      441
Timor-Leste     441
Syria           441
Name: Country/Region, dtype: int64

In [11]:
full_table[full_table['Recovered'].isna()]['Date'].value_counts()

3/31/21    21
6/26/20    21
8/10/20    21
6/14/20    21
8/15/20    21
           ..
1/23/21    21
6/23/20    21
7/23/20    21
3/2/21     21
8/24/20    21
Name: Date, Length: 441, dtype: int64

In [12]:
full_table['Recovered'] = full_table['Recovered'].fillna(0)
full_table['Recovered'] = full_table['Recovered'].astype('int')
full_table.isna().sum()

Province/State    83349
Country/Region        0
Lat                 441
Long                441
Date                  0
Confirmed             0
Deaths                0
Recovered             0
dtype: int64

# Preprocessing

In [13]:
# renaming
# ========

# renaming countries, regions, provinces
full_table['Country/Region'] = full_table['Country/Region'].replace('Korea, South', 'South Korea')

In [14]:
# removing
# =======

# removing canada's recovered values
full_table = full_table[full_table['Province/State'].str.contains('Recovered')!=True]

# removing county wise data to avoid double counting
full_table = full_table[full_table['Province/State'].str.contains(',')!=True]

# Fixing off data

In [15]:
# new values
feb_12_conf = {'Hubei' : 34874}

In [16]:
# function to change value
def change_val(date, ref_col, val_col, dtnry):
    for key, val in dtnry.items():
        full_table.loc[(full_table['Date']==date) & (full_table[ref_col]==key), val_col] = val

In [17]:
# changing values
change_val('2/12/20', 'Province/State', 'Confirmed', feb_12_conf)

In [18]:
# checking values
full_table[(full_table['Date']=='2/12/20') & (full_table['Province/State']=='Hubei')]

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
5825,Hubei,China,30.9756,112.2707,2/12/20,34874,1068,2686


# Saving final data

In [19]:
full_table.to_csv('covid_19_clean_complete.csv', index=False)